In [21]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
import ipympl

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_derived = '/Users/alina/Desktop/MIT/code/data/output/derived_data'
os.listdir(data_derived)

['.DS_Store',
 'original_and_new_rr_14.csv',
 'original_and_new_rr_14_contrasts.csv',
 'original_rr_results_14.csv',
 'treatment_groups.csv']

### Mediators 
- prior treatment with psychoactive medication, 
- ODD/CD comorbidity  (parent, DIS version 3.0), 
- comorbid anx (parent, DIS version 2.3), 
- familial reciept of public assistance
### Moderators 
- as intended (M) : 
  - acceptance of treatment 
  - family attendance of 80% monthly medication visit 
  - prescription written and deluvered to family at visit 
- below intended 
  - refusal, stop treatment, moving, droping out prior to 80% of visits, or attendance below cutoff 
- as intended (B) : 
  - acceptance of treatment 
  - attendance 75% group parent training sessions 
  - child attendance 74% summer treatment program 
  - child + paraproffessional present 75% in classroom at intended days 
- below intended :
  - refusal of any treatment component, 
  - low attendance, 
  - dicontinuation before cutoffs 
- as intended (C): 
  - all criteria for M and B 

In [23]:
snap_file = 'snap01.txt'
ssrs_file = 'ssrs01.txt'
masc_file = 'masc_p01.txt'
parent_child_file = 'pcrc01.txt'
wechsler_file = 'wiat_iiip201.txt'
treat_group = 'treatment_groups.csv'

snap = pd.read_csv(Path(data_root, snap_file), delimiter="\t")
ssrs = pd.read_csv(Path(data_root, ssrs_file), delimiter="\t")
masc = pd.read_csv(Path(data_root, masc_file), delimiter="\t")
pc = pd.read_csv(Path(data_root, parent_child_file), delimiter="\t")
wechsler = pd.read_csv(Path(data_root, wechsler_file), delimiter="\t")
treat_group = pd.read_csv(Path(data_derived, treat_group))


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_8125/3621943440.py:8: DtypeWarning: Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,76,77,78,79,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158) have mixed types. Specify dtype option on import or set low_memory=False.
  snap = pd.read_csv(Path(data_root, snap_file), delimiter="\t")
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_8125/3621943440.py:9: DtypeWarning: Columns (0,1,2,6,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,5

In [24]:
files_diags = ['diagpsx01', 'diagpsx_p201' ,'diagpsx_p301' ,'diagpsx_p401' ,'diagpsx_p501', 'diagpsx_p601', 'diagpsx_p701']

trt_name = pd.read_csv(Path(data_derived, 'treatment_groups.csv'))
services_file = 'scapijus01.txt'
meds_file = 'medse01.txt'
expectations_file = 'expect01.txt'
initial_percieved_satisfaction_file = 'debrief01.txt'
int_dsm4_file = 'scid01.txt'
diags1_file  = 'diagpsx01.txt'

beck_dep_file = 'bdi01.txt'
beck_dep= pd.read_csv(Path(data_root, beck_dep_file), delimiter='\t')


diags = []

for file in files_diags:
    diags.append(pd.read_csv(Path(data_root, file + '.txt'), delimiter='\t'))
    
    
end_treat_file = 'debrief1401.txt'
end_treat = pd.read_csv(Path(data_root, end_treat_file), delimiter='\t')

demog_file = 'demgr01.txt'
demog = pd.read_csv(Path(data_root, demog_file), delimiter= '\t')

health_qst_file  = 'health01.txt'
health_qst = pd.read_csv(Path(data_root, health_qst_file), delimiter='\t')

services = pd.read_csv(Path(data_root, services_file), delimiter='\t')
meds = pd.read_csv(Path(data_root, meds_file), delimiter='\t')

dsm4 = pd.read_csv(Path(data_root, int_dsm4_file), delimiter='\t')
diags1 = pd.read_csv(Path(data_root, diags1_file), delimiter='\t')

expect = pd.read_csv(Path(data_root, expectations_file), delimiter='\t')
initial_sat = pd.read_csv(Path(data_root, initial_percieved_satisfaction_file), delimiter='\t')

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_8125/85365958.py:12: DtypeWarning: Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167) have mixed types. Specify dtype option on import or set low_memory=False.
  beck_dep= pd.read_csv(Path(data_root, beck_dep_file), delimiter='\t')
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_8125/85365958.py:18: DtypeWarning: Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,

## Comorbidities Anx and ODD/CD

19 pso Social phobia (13093,) diagpsx01
20 psoi Social phobia with impairment (12903,) diagpsx01
21 pag Agoraphobia (13086,) diagpsx01
22 pagi Agoraphobia with impairment (14475,) diagpsx01
25 pga Generalized anxiety disorder (13081,) diagpsx01
26 pgai Generalized anxiety disorder with impairment (14459,) diagpsx01
27 poa Overanxious disorder (5173,) diagpsx01
28 poai Overanxious disorder with impairment (5173,) diagpsx01
30 psa Separation anxiety disorder (9740,) diagpsx01
31 psai Separation anxiety disorder with impairment (9741,) diagpsx01

In [25]:
prep.print_nonNaN_shapes(beck_dep)

0 collection_id collection_id (7194,)
1 bdi01_id bdi01_id (7194,)
2 dataset_id dataset_id (7194,)
3 subjectkey The NDAR Global Unique Identifier (GUID) for research subject (7194,)
4 src_subject_id Subject ID how it's defined in lab/project (7194,)
5 interview_date Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY (7194,)
6 interview_age Age in months at the time of the interview/test/sampling/imaging. (7194,)
7 sex Sex of subject at birth (7194,)
9 bdi_sad Sadness (7185,)
10 bdi_pess Pessimism (7183,)
11 bdi_pf Past failure (7188,)
12 bdi_lop Loss of pleasure (7176,)
13 bdi_gf Guilty feelings (7182,)
14 bdi_punf Punishment feelings (7188,)
15 bdi_sd Self-dislike (7184,)
16 bdi_sc Self-criticalness (7179,)
17 bdi_stw Suicidal thoughts or wishes (7193,)
18 bdy_cry Crying (7187,)
20 bdi_li Loss of interest (7191,)
21 bdi_indec Indecisiveness (7190,)
26 bdi_irrit Irritability (7187,)
30 bdi_tf Tiredness or fatigue (7177,)
31 bdi_lis Loss of in

In [26]:
baseline_vars = var_dict.get_baseline_vars()
baseline_vars

['src_subject_id',
 'interview_date',
 'interview_age',
 'sex',
 'site',
 'days_baseline',
 'relationship']

In [59]:
baseline_vars

['src_subject_id',
 'interview_date',
 'interview_age',
 'sex',
 'site',
 'days_baseline',
 'relationship']

In [27]:
0
# for now remove poa because lots of mssing data in comparison 
vars_comorb = np.concatenate((baseline_vars, var_oddrcd, vars_anx))
comorb__ = diags1[vars_comorb]
comorb_ = pd.merge(comorb__, trt_name, how='inner', on='src_subject_id')

In [28]:
vars_comorb_types = np.concatenate((var_oddrcd, vars_anx))
vars_comorb_type_dict = dict(zip(vars_comorb_types, ['category'] * len(vars_anx)))

comorb= prep.set_dtypes(comorb_, vars_comorb_type_dict, set_baseline=True)
comorb.dtypes

Conversion encountered a problem. Attempt to drop description line.
Success


src_subject_id            object
interview_date    datetime64[ns]
interview_age              int64
sex                     category
site                    category
days_baseline              int64
relationship              object
cdorodd                 category
pso                     category
psoi                    category
pag                     category
pagi                    category
pga                     category
pgai                    category
psa                     category
psai                      object
trtname                 category
dtype: object

## Public assistance 

In [29]:
services

,collection_id,scapijus01_id,dataset_id,subjectkey,src_subject_id,interview_date,interview_age,sex,monthsbl,daystrt,...,glhitnum,glhitpc,glalthrs,glaltnm,glaladtn,glalldtn,glalbetn,version_form,asssca,collection_title
0,collection_id,scapijus01_id,dataset_id,The NDAR Global Unique Identifier (GUID) for r...,Subject ID how it's defined in lab/project,Date on which the interview/genetic test/sampl...,Age in months at the time of the interview/tes...,Sex of subject at birth,Actual number of months from Baseline,Parent SNAP or CAARS Days from Baseline used a...,...,Number of times during year received most inte...,Percent of year received most intensive therapy,All therapies combined: total amount of hours ...,All therapies combined: number of therapies re...,All therapies combined: number of therapies re...,All therapies combined: number of therapies re...,All therapies combined: number of therapies re...,Form used/assessment name,Assessment point,collection_title
1,2155,83322,9363,NDAR_INVDW676PU1,P1561,02/25/2002,177,F,80,2454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIMON,84,Multimodal Treatment Study of Children With ADHD
2,2155,83323,9363,NDAR_INVDW676PU1,P1561,01/06/2003,187,F,90,2769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIMON,96,Multimodal Treatment Study of Children With ADHD
3,2155,83324,9363,NDAR_INVDW676PU1,P1561,11/18/2003,197,F,101,3085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIMON,108,Multimodal Treatment Study of Children With ADHD
4,2155,83325,9363,NDAR_INVDW676PU1,P1561,09/03/2004,207,F,110,3375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIMON,120,Multimodal Treatment Study of Children With ADHD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27304,2155,83317,9364,NDAR_INVXP709VPA,P1844,01/22/2000,129,M,69,1734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIPMD,72,Multimodal Treatment Study of Children With ADHD
27305,2155,83318,9364,NDAR_INVXP709VPA,P1844,02/01/2001,141,M,80,2110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIPMD,84,Multimodal Treatment Study of Children With ADHD
27306,2155,83319,9364,NDAR_INVXP709VPA,P1844,01/21/2002,153,M,92,2464,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIPMD,96,Multimodal Treatment Study of Children With ADHD
27307,2155,83320,9364,NDAR_INVXP709VPA,P1844,01/10/2003,165,M,104,2818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIPMD,108,Multimodal Treatment Study of Children With ADHD


In [30]:
# file = services
# for i, col in zip(range(file.shape[1]), file.columns): 
#     if file.iloc[:,i].dropna().shape[0] > 1 :#and (file.iloc[0, i].rfind('anx') != -1 or  file.iloc[0, i].rfind('cond') != -1 or file.iloc[0, i].rfind('opp') != -1 or file.iloc[0, i].rfind('pho') != -1 ) : 
#         print(i,col, file.iloc[0, i], file.iloc[:,i].dropna().shape)

In [31]:
#rep.print_nonNaN_shapes(demog)

In [32]:
assist_vars_ = ['demo61']
assist_vars_dtype = ['category'] # change when found out what the values are, now 1 and 2
assist_vars_dtype_dict = dict(zip(assist_vars_, assist_vars_dtype))

assist_vars = np.concatenate((baseline_vars, assist_vars_))
assist__ = demog[assist_vars]
assist_ = pd.merge(assist__, trt_name, how='inner', on ='src_subject_id')
assist = prep.set_dtypes(assist_, assist_vars_dtype_dict, set_baseline=True)

Conversion encountered a problem. Attempt to drop description line.
Success


In [33]:
assist.dtypes

src_subject_id            object
interview_date    datetime64[ns]
interview_age              int64
sex                     category
site                    category
days_baseline              int64
relationship              object
demo61                  category
trtname                 category
dtype: object

## Previous medication


In [34]:
#prep.print_nonNaN_shapes(meds)

In [35]:
#prep.print_nonNaN_shapes(health_qst, contains='med')

In [36]:
#prep.print_nonNaN_shapes(demog, contains='med')

In [37]:
prev_med_vars_ = ['hi_24']
prev_med_dytpes = ['category']
prev_med_dtype_dict = dict(zip(prev_med_vars_, prev_med_dytpes))
prev_med_vars = np.concatenate((baseline_vars, prev_med_vars_))
prev_med__ = health_qst[prev_med_vars]
prev_med_ = pd.merge(prev_med__, trt_name, how='inner', on = 'src_subject_id')

prev_med= prep.set_dtypes(prev_med_, prev_med_dtype_dict, set_baseline=True)
prev_med.dtypes

Conversion encountered a problem. Attempt to drop description line.
Success


src_subject_id            object
interview_date    datetime64[ns]
interview_age              int64
sex                     category
site                    category
days_baseline              int64
relationship              object
hi_24                   category
trtname                 category
dtype: object

In [38]:
prev_med

,src_subject_id,interview_date,interview_age,sex,site,days_baseline,relationship,hi_24,trtname
1,P1001,2005-10-12,256,M,1,3029,16,NaN,L
2,P1001,2007-12-29,282,M,1,3836,16,1,L
3,P1001,2010-08-04,313,M,1,4786,16,1,L
4,P1001,2011-12-29,330,M,1,5297,16,1,L
5,P1002,2010-12-14,282,M,1,4942,16,2,C
...,...,...,...,...,...,...,...,...,...
2823,P1867,2011-07-17,258,M,4,4919,16,1.0,L
2824,P1868,2006-08-09,198,F,1,3114,1,NaN,L
2825,P1868,2008-07-23,222,F,1,3828,16,1.0,L
2826,P1868,2010-09-30,248,F,1,4627,16,1.0,L


## Initial acceptance 

In [39]:
#prep.print_nonNaN_shapes(expect)

In [40]:
expect['actbdic'].unique()

array(['Active status', '0', '1', '2', '3'], dtype=object)

In [41]:
#prep.print_nonNaN_shapes(initial_sat)

In [42]:
print(initial_sat['d2dresp'].unique())

['Which of the following choices best describes the responses of the parent(s) to randomization?'
 nan '3' '4' '6' '5' '1' '2']


In [43]:
#prep.print_nonNaN_shapes(end_treat)

In [44]:
accept_vars_ = ['d2dresp']
accept_vars_dtypes = ['category']
accept_vars_dtypes_dict = dict(zip(accept_vars_, accept_vars_dtypes))
accept_vars = np.concatenate((baseline_vars, accept_vars_))

In [45]:
accept__ = initial_sat[accept_vars]
accept_ = pd.merge(accept__, trt_name, how='inner', on='src_subject_id')
accept = prep.set_dtypes(accept_, accept_vars_dtypes_dict, set_baseline=True)

Conversion encountered a problem. Attempt to drop description line.
Success


In [46]:
accept.dtypes

src_subject_id            object
interview_date    datetime64[ns]
interview_age              int64
sex                     category
site                    category
days_baseline              int64
relationship              object
d2dresp                 category
trtname                 category
dtype: object

### Adherence - missing 

In [47]:
print(comorb.shape, prev_med.shape, assist.shape, accept.shape)
print(comorb.dropna().shape, prev_med.dropna().shape, assist.dropna().shape, accept.dropna().shape)
print(comorb['src_subject_id'].unique().shape, prev_med['src_subject_id'].unique().shape, assist['src_subject_id'].unique().shape, accept['src_subject_id'].unique().shape)

(9333, 17) (2827, 9) (4076, 9) (579, 9)
(2545, 17) (2012, 9) (2055, 9) (263, 9)
(868,) (762,) (798,) (579,)


## Save 

In [48]:
# comorb.to_csv(Path(data_derived, 'comorb.csv'))
# prev_med.to_csv(Path(data_derived, 'prev_med.csv'))
# assist.to_csv(Path(data_derived, 'assist.csv'))
# accept.to_csv(Path(data_derived, 'accept.csv'))


In [49]:
prep.print_nonNaN_shapes(services, contains='med')

137 monyn Were there any ADHD medication monitoring visits during this interval? (588,)
138 monprov1 Medication monitoring provider 1 (575,)
141 monprov2 Medication monitoring provider 2 (100,)
144 monprov3 Medication monitoring provider 3 (18,)
147 monprov4 Medication monitoring provider 4 (4,)
150 monvist Total number of medication monitoring visits during this interval (1717,)
151 monprot Number of different provider types seen for medication monitoring during this interval (1717,)
152 monhrst Total number of hours of medication monitoring during this interval (1717,)
153 monvisc Cumulative number of medication monitoring visits (1717,)
154 monhrsc Cumulative number of medication monitoring hours (1717,)
155 paryn Did a parent take psychiatric medication this interval? (5382,)
156 medrel If yes, which parent took this psychiatric medication? (1852,)
157 adhd_mta Was the psychiatric medication for ADHD? (387,)
158 anx_mta Was the psychiatric medication for Anxiety? (420,)
159 dep_mta

In [52]:

prep.print_nonNaN_shapes(expect, contains='columb')

In [53]:
columbia_file = 'cisp_p01.txt'
columbia = pd.read_csv(Path(data_root, columbia_file), delimiter='\t')

In [54]:
prep.print_nonNaN_shapes(services, contains='med')

137 monyn Were there any ADHD medication monitoring visits during this interval? (588,)
138 monprov1 Medication monitoring provider 1 (575,)
141 monprov2 Medication monitoring provider 2 (100,)
144 monprov3 Medication monitoring provider 3 (18,)
147 monprov4 Medication monitoring provider 4 (4,)
150 monvist Total number of medication monitoring visits during this interval (1717,)
151 monprot Number of different provider types seen for medication monitoring during this interval (1717,)
152 monhrst Total number of hours of medication monitoring during this interval (1717,)
153 monvisc Cumulative number of medication monitoring visits (1717,)
154 monhrsc Cumulative number of medication monitoring hours (1717,)
155 paryn Did a parent take psychiatric medication this interval? (5382,)
156 medrel If yes, which parent took this psychiatric medication? (1852,)
157 adhd_mta Was the psychiatric medication for ADHD? (387,)
158 anx_mta Was the psychiatric medication for Anxiety? (420,)
159 dep_mta

In [55]:
services[['src_subject_id', 'daystrt', 'monyn']].dropna()

,src_subject_id,daystrt,monyn
0,Subject ID how it's defined in lab/project,Parent SNAP or CAARS Days from Baseline used a...,Were there any ADHD medication monitoring visi...
17,P1564,0,1
18,P1564,418,1
19,P1564,683,1
40,P1566,430,1
...,...,...,...
1711,P1844,1358,1.0
1712,P1844,1734,1.0
1714,P1844,2464,1.0
1715,P1844,2818,1.0


In [56]:
for cols in meds.iloc[:, 9:17].columns:
    print(meds[cols].dropna().shape)

(3432,)
(3412,)
(3412,)
(3063,)
(3215,)
(3219,)
(3255,)
(44,)


In [57]:
meds.iloc[:, 9:17]

,trtname,mmvnum,visityn,mmday,mmdrug,mmdose,mmdaily,asispme
0,Treatment group,Maintenance Medication Visit Number,Did the Maintenance Medication Visit take place?,Maintenance Medication Visit - Days from Basel...,Name of Medication Prescribed at this Visit (i...,Dosage Regimen of Above Medication Prescribed ...,Total Daily Dose of Above Medication (in mg),Specify medication by name
1,C,1,1,66,No drug,0,0,NaN
2,C,2,1,101,No drug,0,0,NaN
3,C,3,1,135,No drug,0,0,NaN
4,C,4,1,156,No drug,0,0,NaN
...,...,...,...,...,...,...,...,...
3427,M,2,1,139,Ritalin,20x20x10,50,NaN
3428,M,3,1,168,Dexedrine,5srx0x0,5,NaN
3429,M,4,1,201,Dexedrine,20srx0x5,25,NaN
3430,M,5,1,230,Dexedrine,20srx0x5,25,NaN
